<a href="https://colab.research.google.com/github/leoxiang66/CS-285-at-UC-Berkeley-Deep-Reinforcement-Learning-2021/blob/main/examples/reinforce_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/leoxiang66/CS-285-at-UC-Berkeley-Deep-Reinforcement-Learning-2021.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/leoxiang66/CS-285-at-UC-Berkeley-Deep-Reinforcement-Learning-2021.git to /tmp/pip-req-build-cedzz5kk
  Running command git clone -q https://github.com/leoxiang66/CS-285-at-UC-Berkeley-Deep-Reinforcement-Learning-2021.git /tmp/pip-req-build-cedzz5kk
  Created wheel for RLalgos: filename=RLalgos-0.1.4-py3-none-any.whl size=8159 sha256=c718dc95240b940619a5b1d0145af9149965d32d1cc4d8a97f45b66807230aa9
  Stored in directory: /tmp/pip-ephem-wheel-cache-zc0boeog/wheels/4c/56/f2/8ddd79049992cb6f5396813f1f0ab97eac7ab2633c7c9eb79d
Successfully built RLalgos
  Attempting uninstall: RLalgos
    Found existing installation: RLalgos 0.1.3
    Uninstalling RLalgos-0.1.3:
      Successfully uninstalled RLalgos-0.1.3


In [2]:
from RLalgos import reinforce
import gym
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical
from collections import deque
import numpy as np

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
######################### environment ##########################
env_id = "CartPole-v1"
# Create the env
env = gym.make(env_id)

# Create the evaluation env
eval_env = gym.make(env_id)

# Get the state space and action space
s_size = env.observation_space.shape[0]
a_size = env.action_space.n

print("_____OBSERVATION SPACE_____ \n")
print("The State Space is: ", s_size)
print("Sample observation", env.observation_space.sample()) # Get a random observation

print("\n _____ACTION SPACE_____ \n")
print("The Action Space is: ", a_size)
print("Action Space Sample", env.action_space.sample()) # Take a random action

_____OBSERVATION SPACE_____ 

The State Space is:  4
Sample observation [-4.1055722e+00  8.9001972e+37  2.1976815e-01 -1.6123223e+38]

 _____ACTION SPACE_____ 

The Action Space is:  2
Action Space Sample 0


/usr/local/lib/python3.7/dist-packages/gym/core.py:318: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  "Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future."
/usr/local/lib/python3.7/dist-packages/gym/wrappers/step_api_compatibility.py:40: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  "Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future."


In [4]:
######################## policy #################################
class Policy(nn.Module):
    def __init__(self, s_size, a_size, h_size):
        super(Policy, self).__init__()
        # Create two fully connected layers
        self.fc1 = nn.Linear(s_size, h_size)
        self.fc2 = nn.Linear(h_size, a_size)

    def forward(self, x):
        # Define the forward pass
        # state goes to fc1 then we apply ReLU activation function
        # fc1 outputs goes to fc2
        # We output the softmax

        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x, dim=1)

    def act(self, state):
        """
        Given a state, take action
        """
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        probs = self.forward(state).cpu()
        m = Categorical(probs)
        action = m.sample()
        return action.item(), m.log_prob(action)


In [6]:

########################## training #############################
cartpole_hyperparameters = {
    "h_size": 16,
    "n_training_episodes": 1000,
    "n_evaluation_episodes": 10,
    "max_t": 1000,
    "gamma": 1.0,
    "lr": 1e-2,
    "env_id": env_id,
    "state_space": s_size,
    "action_space": a_size,
}

# Create policy and place it to the device
cartpole_policy = Policy(cartpole_hyperparameters["state_space"], cartpole_hyperparameters["action_space"], cartpole_hyperparameters["h_size"]).to(device)
cartpole_optimizer = optim.Adam(cartpole_policy.parameters(), lr=cartpole_hyperparameters["lr"])

scores = reinforce(cartpole_policy,
                   cartpole_optimizer,
                   cartpole_hyperparameters["n_training_episodes"],
                   cartpole_hyperparameters["max_t"],
                   cartpole_hyperparameters["gamma"],
                   100,
                   env)

#################################################################

Episode 100	Average Score: 22.12
Episode 200	Average Score: 54.87
Episode 300	Average Score: 63.88
Episode 400	Average Score: 70.27
Episode 500	Average Score: 80.27
Episode 600	Average Score: 71.05
Episode 700	Average Score: 74.84
Episode 800	Average Score: 65.15
Episode 900	Average Score: 71.18
Episode 1000	Average Score: 57.10


In [7]:
###################### evaluation ###############################
def evaluate_agent(env, max_steps, n_eval_episodes, policy):
    """
    Evaluate the agent for ``n_eval_episodes`` episodes and returns average reward and std of reward.
    :param env: The evaluation environment
    :param n_eval_episodes: Number of episode to evaluate the agent
    :param policy: The Reinforce agent
    """
    episode_rewards = []
    for episode in range(n_eval_episodes):
        state = env.reset()
        step = 0
        done = False
        total_rewards_ep = 0

        for step in range(max_steps):
            action, _ = policy.act(state)
            new_state, reward, done, info = env.step(action)
            total_rewards_ep += reward

            if done:
                break
            state = new_state
        episode_rewards.append(total_rewards_ep)
    mean_reward = np.mean(episode_rewards)
    std_reward = np.std(episode_rewards)

    return mean_reward, std_reward

evaluate_agent(eval_env,
               cartpole_hyperparameters["max_t"],
               cartpole_hyperparameters["n_evaluation_episodes"],
               cartpole_policy)

(52.8, 14.722771478223793)